#### Langchain

###### The basic building block of LangChain is a Large Language Model which takes text as input and generates more text
###### Suppose we want to generate a company name based on the company description, so we will first initialize an OpenAI wrapper. In this case, since we want the output to be more random, we will intialize our model with high temprature.
###### The temperature parameter adjusts the randomness of the output. Higher values like 0.7 will make the output more random, while lower values like 0.2 will make it more focused and deterministic.

In [ ]:
import langchain
import os


In [ ]:
Huggingface_Key = os.getenv("HUGGING_FACE_API")
Google_Key = os.getenv("GOOGLE_KEY_API")
OpenAi_Key = os.getenv("OPEN_KEY_API")
SERPAPI_API_KEY = os.getenv("SERPAPI_API_KEY")

##### OpenAI

In [ ]:
from langchain.llms import OpenAI

# temperature value--> how creative we want our model to be
#0 ---> temperature it means model is very safe it is not taking any bets.
#1 --> it will take risk it might generate wrong output but it is very creative

llm = OpenAI(openai_api_key=OpenAi_Key, temperature=0.8)
print(llm.model_name)

text = "what would be a good company name that produces electronic gadgets"
print(llm.invoke(text))

#### Hugging Face

In [11]:
from langchain import HuggingFaceHub

llm2 = HuggingFaceHub(repo_id="google/flan-t5-large",
                      huggingfacehub_api_token=Huggingface_Key, 
                      model_kwargs={"temperature": 0})

llm2("Traslate english to german: My name is pramod?")


d:\GenAi\Models\envAI2\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceHub` was deprecated in LangChain 0.0.21 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEndpoint`.
  warn_deprecated(
d:\GenAi\Models\envAI2\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
d:\GenAi\Models\envAI2\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The method `BaseLLM.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


'Ich bin Pramod?'

##### Prompt Templates

In [24]:
from langchain.prompts import PromptTemplate

llm3 = HuggingFaceHub(repo_id="google/flan-t5-large",
                      huggingfacehub_api_token=Huggingface_Key, 
                      model_kwargs={"temperature": 0})

prompt_template_name = PromptTemplate(
    input_variables=['cuisine'], 
    template="I want to open a restaurant for {cuisine} food and give me some suggestions for the restaurant name"
)

prompt = prompt_template_name.format(cuisine = "Italian")
print(prompt)

I want to open a restaurant for Italian food and give me some suggestions for the restaurant name


#### Chains

In [25]:

prompt1 = PromptTemplate.from_template(
    " I am fan of {product} biryani. Suggest me a good name for my son so that by hearing that name it should remind me"
)
prompt1.format(product="Italian")

' I am fan of Italian biryani. Suggest me a good name for my son so that by hearing that name it should remind me'

In [26]:
from langchain.chains import LLMChain

chain = LLMChain(llm=llm3, prompt=prompt1, verbose=True)  # verbose is for viewing the flow of the chain
response = chain.run("Hyderabad")
print(response)

d:\GenAi\Models\envAI2\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(




> Entering new LLMChain chain...
Prompt after formatting:
 I am fan of Hyderabad biryani. Suggest me a good name for my son so that by hearing that name it should remind me

> Finished chain.
hyderabad


#### Simple Sequential Chain

In [27]:
llm4 = HuggingFaceHub(repo_id="google/flan-t5-large",
                      huggingfacehub_api_token=Huggingface_Key, 
                      model_kwargs={"temperature": 0.8})

prompt_template_name = PromptTemplate(
    input_variables=['cuisine'], 
    template="I want to open a restaurant for {cuisine} food and give me some suggestions for the restaurant name"
)

chain_cuisine = LLMChain(llm=llm4, prompt=prompt1, verbose=True, output_key="restaurant name")

prompt_template_name1 = PromptTemplate(
    input_variables=['restaurant name'], 
    template="Suggest 10 menu items for {restaurant name}"
)

chain_name = LLMChain(llm = llm4, prompt=prompt_template_name1, verbose=True, output_key="menu items")

In [28]:
from langchain.chains import SimpleSequentialChain
chain2 = SimpleSequentialChain(
    chains = [chain_cuisine, chain_name]
)

content = chain2.run("indian")
print(content)



> Entering new LLMChain chain...
Prompt after formatting:
 I am fan of indian biryani. Suggest me a good name for my son so that by hearing that name it should remind me

> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
Suggest 10 menu items for samantha

> Finished chain.
grilled chicken, grilled asparagus, grilled chicken, grilled chicken, grilled asparagus


#### Sequential Chain

In [29]:
llm4 = HuggingFaceHub(repo_id="google/flan-t5-large",
                      huggingfacehub_api_token=Huggingface_Key, 
                      model_kwargs={"temperature": 0.8})

prompt_template_name = PromptTemplate(
    input_variables =['cuisine'],
    template = "I want to open a restaurant for {cuisine} food. Suggest a fency name for this."
)

name_chain =LLMChain(llm=llm4, prompt=prompt_template_name, output_key="restaurant_name", verbose=True)

prompt_template_items = PromptTemplate(
    input_variables = ['restaurant_name'],
    template="Suggest some menu items for {restaurant_name}."
)

food_items_chain =LLMChain(llm=llm4, prompt=prompt_template_items, output_key="menu_items", verbose=True)

In [30]:
from langchain.chains import SequentialChain

chain = SequentialChain(
    chains = [name_chain, food_items_chain],
    input_variables = ['cuisine'],
    output_variables = ['restaurant_name', "menu_items"]
)

output = chain({"cuisine":"italian"})
print(output)

d:\GenAi\Models\envAI2\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(




> Entering new LLMChain chain...
Prompt after formatting:
I want to open a restaurant for italian food. Suggest a fency name for this.

> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
Suggest some menu items for io tagliacci.

> Finished chain.
{'cuisine': 'italian', 'restaurant_name': 'io tagliacci', 'menu_items': 'grilled octopus, grilled octopus, grilled '}


#### Agents and Tools

In [45]:
from langchain.agents import AgentType, initialize_agent, load_tools
from langchain_community.utilities import SerpAPIWrapper
from langchain import HuggingFaceHub


llm4 = HuggingFaceHub(repo_id="google/flan-t5-large",
                      huggingfacehub_api_token=Huggingface_Key, 
                      model_kwargs={"temperature": 0.8})

#serpapi = SerpAPIWrapper(serpapi_api_key=SERPAPI_API_KEY)

#Google Search API
# The tools we'll give the Agent access to. Note that the 'llm-math' tool uses an LLM, so we need to pass that in.
tools = load_tools(["wikipedia", "llm-math"], llm=llm4)

# Finally, let's initialize an agent with the tools, the language model, and the type of agent we want to use.
agent = initialize_agent(tools,
                        llm4,
                        agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, 
                        verbose=True,handle_parsing_errors=True)

# Let's test it out!
agent.run("What was the GDP of india in 2023?")



> Entering new AgentExecutor chain...
[indian economy, gdp]
Observation: Invalid Format: Missing 'Action:' after 'Thought:
Thought:[indian economy, gdp]
Observation: Invalid Format: Missing 'Action:' after 'Thought:
Thought:[indian economy, gdp]
Observation: Invalid Format: Missing 'Action:' after 'Thought:
Thought:[indian economy, gdp] Observation: Invalid Format: Miss
Observation: Invalid Format: Missing 'Action:' after 'Thought:
Thought:[indian economy, gdp] Observation: Invalid Format: Miss
Observation: Invalid Format: Missing 'Action:' after 'Thought:
Thought:[indian economy, gdp] Observation: Invalid Format: Miss
Observation: Invalid Format: Missing 'Action:' after 'Thought:
Thought:[indian economy, gdp] Observation: Invalid Format: Miss
Observation: Invalid Format: Missing 'Action:' after 'Thought:
Thought:[indian economy, gdp] Observation: Invalid Format: Miss
Observation: Invalid Format: Missing 'Action:' after 'Thought:
Thought:[indian economy, gdp] Observation: Invalid For

'Agent stopped due to iteration limit or time limit.'

#### Memory

In [47]:
llm4 = HuggingFaceHub(repo_id="google/flan-t5-large",
                      huggingfacehub_api_token=Huggingface_Key, 
                      model_kwargs={"temperature": 0.8})

prompt_template_name = PromptTemplate(
    input_variables =['cuisine'],
    template = "I want to open a restaurant for {cuisine} food. Suggest a fency name for this."
)

name_chain =LLMChain(llm=llm4, prompt=prompt_template_name, output_key="restaurant_name", verbose=True)
name = name_chain.run("Mexican")
print(name)



> Entering new LLMChain chain...
Prompt after formatting:
I want to open a restaurant for Mexican food. Suggest a fency name for this.

> Finished chain.
Mexican restaurant


In [48]:
name2 = name_chain.run("indian")
print(name2)



> Entering new LLMChain chain...
Prompt after formatting:
I want to open a restaurant for indian food. Suggest a fency name for this.

> Finished chain.
Indian restaurant


In [49]:
type(name_chain.memory)

NoneType

#### Conversation Buffer Memory

In [50]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory()

chain = LLMChain(llm=llm4, prompt=prompt_template_name, memory=memory)
name = chain.run("Mexican")
print(name)

Mexican restaurant


In [53]:
name2 = chain.run("indian")
print(name2)

print(chain.memory.buffer)

Indian restaurant
Human: Mexican
AI: Mexican restaurant
Human: indian
AI: Indian restaurant


#### ConversationChain

###### Conversation buffer memory goes growing endlessly
###### Just remember last 5 Conversation Chain
###### Just remember last 10-20 Conversation Chain

In [55]:
from langchain.chains import ConversationChain

convo = ConversationChain(llm=llm4)
print(convo.prompt.template)

The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
{history}
Human: {input}
AI:


d:\GenAi\Models\envAI2\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `ConversationChain` was deprecated in LangChain 0.2.7 and will be removed in 1.0. Use RunnableWithMessageHistory: https://api.python.langchain.com/en/latest/runnables/langchain_core.runnables.history.RunnableWithMessageHistory.html instead.
  warn_deprecated(


In [56]:
convo.run("Who won the first cricket world cup?")

'The first cricket world cup was won by India.'

In [57]:
convo.run("How much is 5+5?")

'5+5 is a number that is repeated indefinitely.'

In [58]:
convo.run("Who was the captain ofthe winning team?")

'The captain of the winning team was Virat Kohli.'

In [59]:
print(convo.memory.buffer)

Human: Who won the first cricket world cup?
AI: The first cricket world cup was won by India.
Human: How much is 5+5?
AI: 5+5 is a number that is repeated indefinitely.
Human: Who was the captain ofthe winning team?
AI: The captain of the winning team was Virat Kohli.


#### Conversation Buffer Window Memory

In [64]:
from langchain.memory import ConversationBufferWindowMemory

memory = ConversationBufferWindowMemory(k=2)  # K value is  to remember the no of contexts given by the user

convo = ConversationChain(
    llm=llm4,
    memory=memory
)
convo.run("Who won the first cricket world cup?")

'The first cricket world cup was won by India.'

In [65]:
convo.run("How much is 5+5?")

'5+5 is a number that is repeated indefinitely.'

In [66]:
convo.run("Who was the captain of the winning team?")

'The captain of the winning team was Virat Kohli.'

#### Document Loader

In [68]:
from langchain.document_loaders import PyPDFLoader

loader = PyPDFLoader("simple.pdf")
pages = loader.load()

pages

[Document(metadata={'source': 'simple.pdf', 'page': 0}, page_content='TYPEOriginalResearch\nPUBLISHED 01 June 2023\nDOI10.3389/fenrg.2023.1172176\nOPENACCESS\nEDITEDBY\nWen Zhong Shen,\nYangzhou University, China\nREVIEWEDBY\nAnitha Gopalan,\nSaveetha University, India\nS. Jenoris Muthiya,\nDayananda Sagar College of\nEngineering, India\n*CORRESPONDENCE\nEl-Sayed M. El-Kenawy,\nSkenawy@ieee.org\nRECEIVED 23 February 2023\nACCEPTED 16 May 2023\nPUBLISHED 01 June 2023\nCITATION\nAlhussan AA, M. El-Kenawy E-S,\nAbdelhamid AA, Ibrahim A, Eid MM and\nKhafaga DS (2023), Wind speed\nforecasting using optimized bidirectional\nLSTM based on dipper throated and\ngenetic optimization algorithms.\nFront. Energy Res. 11:1172176.\ndoi: 10.3389/fenrg.2023.1172176\nCOPYRIGHT\n© 2023 Alhussan, M. El-Kenawy,\nAbdelhamid, Ibrahim, Eid and Khafaga.\nThis is an open-access article distributed\nunder the terms of the Creative\nCommons Attribution License (CC BY) .\nThe use, distribution or reproduction in\n